<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация,-очистка-от-лишних-символов-и-пробелов" data-toc-modified-id="Лемматизация,-очистка-от-лишних-символов-и-пробелов-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация, очистка от лишних символов и пробелов</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Создание-выборок" data-toc-modified-id="Создание-выборок-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Создание выборок</a></span></li><li><span><a href="#Создание-корпусов-слов-и-установка-стоп-слов" data-toc-modified-id="Создание-корпусов-слов-и-установка-стоп-слов-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Создание корпусов слов и установка стоп-слов</a></span></li><li><span><a href="#Векторизация" data-toc-modified-id="Векторизация-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Векторизация</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBM" data-toc-modified-id="LGBM-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>LGBM</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>TF-IDF</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBM" data-toc-modified-id="LGBM-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>LGBM</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.4.3"><span class="toc-item-num">2.4.3&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import math
import numpy as np
import pandas as pd
import re
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import f1_score,classification_report
from tqdm import tqdm
import lightgbm as lgb
import transformers
from tqdm import notebook
from catboost import CatBoostClassifier
from xgboost import XGBRegressor

In [2]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


In [8]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


### Лемматизация, очистка от лишних символов и пробелов

In [9]:
def preprocessing(text):
    tokenized = nltk.word_tokenize(text)
    joined = ' '.join(tokenized)
    texted = re.sub(r'[^a-zA-Z]', ' ', joined)
    resulted = ' '.join(texted.split())
    return resulted

In [10]:
data['lemm_text'] = data['text'].apply(preprocessing)

In [11]:
lemmatizer = WordNetLemmatizer()

def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):

    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])), nltk_tagged)
    lemmatized_sentence = []
    
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:        
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [12]:
tqdm.pandas()
data['lemm_text'] = data['text'].progress_apply(lemmatize_sentence) 

100%|██████████| 159292/159292 [09:39<00:00, 274.69it/s]


In [13]:
data.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,D'aww ! He match this background colour I 'm s...
2,2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man , I 'm really not try to edit war . It..."
3,3,"""\nMore\nI can't make any real suggestions on ...",0,`` More I ca n't make any real suggestion on i...
4,4,"You, sir, are my hero. Any chance you remember...",0,"You , sir , be my hero . Any chance you rememb..."


## Обучение

### Создание выборок

In [14]:
features = data['lemm_text']
target = data['toxic']
features_train,features_test,target_train,target_test=train_test_split(features,target, test_size=0.2,random_state=12345)

### Создание корпусов слов и установка стоп-слов

In [15]:
features_train_corpus = features_train.values
features_test_corpus = features_test.values

display(features_train_corpus)

stopwords = set(nltk_stopwords.words('english'))
display(len(stopwords))

array(["Bushranger you 're a GRASS with no sense of humour . Seen the South Park episode Poor and Stupid ? That 's what I be refer to , a comment obviously lose on you . But you 've show you ca n't fight your own battle and have to run cry to mummy - boo-hoo.90.204.13.4",
       "`` Need administrative help I have be block , iniquitously . Read the above request . I try to explain everything there . Hegemonic use of administrative privilege ought to not be overlook . Ask Administrator if I be harass him to such an extent where I need to be block from edit , that too for 48 hr . For goodness ' sake . What happen to do n't bite the newcomer ? Please talk to Bwilkins . See the talk page of he have a knack of be unreasonable with aplomb . here ``",
       "I 'd also like to point out that he have use a third-person plural pronoun and possessive pronoun to refer to one person ( me ) and one person 's ( my ) possession .",
       ...,
       "`` Agreed . We really should try to stick to the 

179

### Векторизация

In [16]:
models=[]
results=[]

count_vect = CountVectorizer(stop_words=stopwords, dtype=np.float32)

bow_features_train = count_vect.fit_transform(features_train_corpus)
bow_features_test = count_vect.transform(features_test_corpus)

display(bow_features_train.shape)

(127433, 158266)

#### LogisticRegression

In [30]:
def logreg(features_train, target_train): #, features_test, target_test):

    logreg = LogisticRegression(random_state=123)
    
    param_grid = {'C': [0.1,1],
                  'solver': ['lbfgs','liblinear'],
                  'penalty': ['l2']}
    
    grid_search_logreg = GridSearchCV(logreg, param_grid, cv=5, n_jobs=-1, scoring='f1')
    
    grid_search_logreg.fit(features_train, target_train)
    
    print(grid_search_logreg.best_params_)
#    print(classification_report(target_test, grid_search_logreg.predict(features_test)))

    print(grid_search_logreg.best_score_)    
    print(grid_search_logreg.best_estimator_)
    
    return grid_search_logreg.best_estimator_

In [18]:
logreg(bow_features_train,target_train) #, bow_features_test, target_test)

{'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.7520322280505028
LogisticRegression(C=1, random_state=123)


#### LGBM

In [19]:
def lgbm(features_train, target_train): #, features_test, target_test):
    
    lgbm_model = lgb.LGBMClassifier(random_state=123)
    
    param_grid = {'max_depth': [5,10],
                  'n_estimators': [5,10],
                  'learning_rate': [5,10]
                 }
    
    grid_search_lgbm = GridSearchCV(lgbm_model, param_grid, cv=5, n_jobs=-1, scoring='f1')
    
    grid_search_lgbm.fit(features_train, target_train)
    
    print(grid_search_lgbm.best_params_)
#    print(classification_report(target_test, grid_search_lgbm.predict(features_test)))
    print(grid_search_lgbm.best_score_)    
    print(grid_search_lgbm.best_estimator_)

In [20]:
lgbm(bow_features_train,target_train) #, bow_features_test, target_test)

{'learning_rate': 10, 'max_depth': 5, 'n_estimators': 5}
0.18318689353718637
LGBMClassifier(learning_rate=10, max_depth=5, n_estimators=5, random_state=123)


#### CatBoost

In [21]:
def catboost(features_train, target_train): #, features_test, target_test):
    
    cat_model = CatBoostClassifier(random_state=123)
    
    param_grid = {'depth': [2,4],
                  'iterations': [5],
                  'learning_rate': [0.1,0.5]}
    
    grid_search_catboost= GridSearchCV(cat_model, param_grid, cv=5, n_jobs=-1, scoring='f1')
    
    grid_search_catboost.fit(features_train, target_train)
    
    print(grid_search_catboost.best_params_)
#    print(classification_report(target_test, grid_search_cat.predict(features_test)))
    print(grid_search_catboost.best_score_)   
    print(grid_search_catboost.best_estimator_)

In [22]:
catboost(bow_features_train,target_train) #, bow_features_test, target_test)

0:	learn: 0.5990722	total: 540ms	remaining: 2.16s
1:	learn: 0.5240639	total: 1.04s	remaining: 1.56s
2:	learn: 0.4672801	total: 1.53s	remaining: 1.02s
3:	learn: 0.4238993	total: 2.03s	remaining: 508ms
4:	learn: 0.3896184	total: 2.52s	remaining: 0us
0:	learn: 0.5960810	total: 538ms	remaining: 2.15s
1:	learn: 0.5227291	total: 1.03s	remaining: 1.55s
2:	learn: 0.4653821	total: 1.51s	remaining: 1.01s
3:	learn: 0.4216683	total: 2.02s	remaining: 504ms
4:	learn: 0.3881733	total: 2.5s	remaining: 0us
0:	learn: 0.5965683	total: 481ms	remaining: 1.92s
1:	learn: 0.5220454	total: 975ms	remaining: 1.46s
2:	learn: 0.4646526	total: 1.46s	remaining: 975ms
3:	learn: 0.4211209	total: 1.95s	remaining: 488ms
4:	learn: 0.3870548	total: 2.44s	remaining: 0us
0:	learn: 0.5991316	total: 487ms	remaining: 1.95s
1:	learn: 0.5247759	total: 1.02s	remaining: 1.53s
2:	learn: 0.4668881	total: 1.52s	remaining: 1.01s
3:	learn: 0.4226036	total: 2.03s	remaining: 507ms
4:	learn: 0.3886750	total: 2.54s	remaining: 0us
0:	learn:

### TF-IDF

In [23]:
tfidf = TfidfVectorizer(stop_words=stopwords)

tfidf_features_train = tfidf.fit_transform(features_train_corpus)
tfidf_features_test = tfidf.transform(features_test_corpus)

display(tfidf_features_train.shape)

(127433, 158266)

#### LogisticRegression

In [24]:
logreg(tfidf_features_train,target_train) #, tfidf_features_test, target_test)

{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
0.7078651395715221
LogisticRegression(C=1, random_state=123, solver='liblinear')


#### LGBM

In [25]:
lgbm(tfidf_features_train,target_train) #, tfidf_features_test, target_test)

{'learning_rate': 5, 'max_depth': 5, 'n_estimators': 5}
0.2479167111661861
LGBMClassifier(learning_rate=5, max_depth=5, n_estimators=5, random_state=123)


#### CatBoost

In [26]:
catboost(tfidf_features_train,target_train) #, tfidf_features_test, target_test)

0:	learn: 0.5992163	total: 684ms	remaining: 2.74s
1:	learn: 0.5248309	total: 1.35s	remaining: 2.03s
2:	learn: 0.4673447	total: 1.99s	remaining: 1.32s
3:	learn: 0.4225759	total: 2.61s	remaining: 652ms
4:	learn: 0.3881826	total: 3.22s	remaining: 0us
0:	learn: 0.5965709	total: 674ms	remaining: 2.7s
1:	learn: 0.5229974	total: 1.28s	remaining: 1.93s
2:	learn: 0.4652857	total: 1.92s	remaining: 1.28s
3:	learn: 0.4211665	total: 2.53s	remaining: 632ms
4:	learn: 0.3871816	total: 3.15s	remaining: 0us
0:	learn: 0.5969978	total: 687ms	remaining: 2.75s
1:	learn: 0.5221054	total: 1.32s	remaining: 1.98s
2:	learn: 0.4671122	total: 1.97s	remaining: 1.32s
3:	learn: 0.4230679	total: 2.62s	remaining: 655ms
4:	learn: 0.3910160	total: 3.24s	remaining: 0us
0:	learn: 0.5972351	total: 671ms	remaining: 2.68s
1:	learn: 0.5222859	total: 1.31s	remaining: 1.96s
2:	learn: 0.4650203	total: 1.93s	remaining: 1.29s
3:	learn: 0.4213213	total: 2.55s	remaining: 638ms
4:	learn: 0.3870000	total: 3.17s	remaining: 0us
0:	learn:

### Тестирование

In [36]:
#model_logistic = LogisticRegression(C=1.0, penalty="l2", solver= 'lbfgs', random_state = 123)
#model_Logistic.fit(bow_features_train,target_train)

#test_f1_score = f1_score(target_test, model_logistic.predict(bow_features_test))

#f1_score_test = logreg(bow_features_test,target_test)

#f1_score_test = f1_score(target_test, .predict(bow_features_test))

best_estimator = logreg(bow_features_train,target_train)

target_predicted = best_estimator.predict(bow_features_test)

f1 = f1_score(target_test, target_predicted)

print("Test F1 score (LogisticRegression):", f1)

{'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.7520322280505028
LogisticRegression(C=1, random_state=123)
Test F1 score (LogisticRegression): 0.7567380787836903


## Выводы

- Выполнена загрузка представленных текстовых данных и необходимых библиотек, загружены стоп-слова для соотвесвтующего языка (Английского)
- Проведена предобработка и анализ данных, текст лемматизирован, очищен от лишних символов и пробелов
- С помощью двух различных методов (векторизация и TF_IDF), подготовлены признаки для машинного обучения
- Обучены три различные модели (LogisticRegression, LGBM и CatBoost) для различных методов препроцессинга
- Наилучшее значение F1 показала модель LogisticRegression на CountVectoriser, это соотвествует критериям, указанных в условии задачи
- BERT не был использован из-за ограниченности вычислительной мощности